# Callbacks

Un callback es un operador que modifica o usa información de un algoritmo en tiempo
de ejecución. Este operador no es un operador genético, es decir, no tiene relación
con los operadores de *mating* que definen el algoritmo, si no más bien es un operador
de conveniencia para ajustar los operadores principales.


In [1]:
import pickle

import numpy as np

from warehouse_allocation.operators.callback import ReportObjectiveBestValuesCallback
from warehouse_allocation.algorithms import solve_chung_problem

# Cargamos data de ejemplo
with open("data", "rb") as ip:
    example_data = pickle.load(ip)

# Resolvemos el problema de Chung adicionando el callback que reporta los objetivos
# Seteamos la cantidad de objetivos con n_obj = 2
result = solve_chung_problem(
    algorithm_name = "NSGA2",
    D=example_data["D"],
    Z=example_data["Z"],
    OCM=example_data["OCM"],
    W=example_data["W"],
    WT=example_data["WT"],
    algorithm_callback=ReportObjectiveBestValuesCallback(n_obj=2),
    iterations = 20,
    pop_size = 50,
    n_offsprings = 10,
    verbose = True
)

n_gen |  n_eval |  n_nds  |     eps      |  indicator  
    1 |      50 |       8 |            - |            -
    2 |      60 |       8 |  0.114779348 |        ideal
    3 |      70 |      10 |  0.030684689 |            f
    4 |      80 |      12 |  0.009813673 |        ideal
    5 |      90 |      12 |  0.042293770 |        ideal
    6 |     100 |       9 |  0.010141948 |            f
    7 |     110 |       9 |  0.074468085 |        ideal
    8 |     120 |      10 |  0.129005836 |        ideal
    9 |     130 |       8 |  0.011983677 |            f
   10 |     140 |       9 |  0.109571145 |        nadir
   11 |     150 |       9 |  0.00000E+00 |            f
   12 |     160 |       9 |  0.00000E+00 |            f
   13 |     170 |      11 |  0.224893233 |        ideal
   14 |     180 |      11 |  0.000064040 |            f
   15 |     190 |      11 |  0.000604380 |            f
   16 |     200 |      10 |  0.022676420 |        ideal
   17 |     210 |      10 |  0.019958895 |      

In [2]:
# Podemos acceder a la información guardad en cada iteración.
# Se tiene un diccionario, donde cada llave corresponde a una función objetivo
# Por ejemplo, la llave F_1, reporta los mejores valores de F_1 en cada iteración, además
# del valor respectivo de F_2. Análogo la llave F_2.
result.algorithm.callback.data

{'F_1': array([[-137170.,   17096.],
        [-138884.,   16739.],
        [-138884.,   16739.],
        [-139032.,   17100.],
        [-139698.,   17897.],
        [-139698.,   17897.],
        [-140965.,   16718.],
        [-143485.,   17973.],
        [-143485.,   17973.],
        [-143485.,   17973.],
        [-143485.,   17973.],
        [-143485.,   17973.],
        [-148593.,   19265.],
        [-148593.,   19265.],
        [-148593.,   19265.],
        [-149120.,   20176.],
        [-149557.,   19623.],
        [-149557.,   19623.],
        [-149557.,   19623.],
        [-150978.,   19957.]]),
 'F_2': array([[-123951.,   11541.],
        [-123951.,   11541.],
        [-123951.,   11541.],
        [-123951.,   11541.],
        [-123951.,   11541.],
        [-123951.,   11541.],
        [-123951.,   11541.],
        [-123951.,   11541.],
        [-123951.,   11541.],
        [-125880.,   11541.],
        [-125880.,   11541.],
        [-125880.,   11541.],
        [-125880.,   115

Un callback que **modifca** el comportamiento del algoritmo es el definido en ``StopMutationAfterNgenCallback``. Este callback detiene el operador de mutación del mating, despues de un número pre fijado de generaciones. La idea de este callback nace en que un principio podría ser bueno explorar muchas soluciones, y luego simplemente mejorar la población general con la genética ya presente.

In [8]:
from warehouse_allocation.operators import ChungAislePermutationMutation
from warehouse_allocation.operators.callback import StopMutationAfterNgenCallback

from pymoo.operators.mutation.nom import NoMutation


# Aplicamos el operador de mutación con una tasa alta (note que la tasa es a elección del usuario)
# y aplicamos el callback que detendrá el operador de mutación luego de la generación 10
# Este callback convertira el operador en NoMutation, que es una dummy class.
result = solve_chung_problem(
    algorithm_name = "NSGA2",
    D=example_data["D"],
    Z=example_data["Z"],
    OCM=example_data["OCM"],
    W=example_data["W"],
    WT=example_data["WT"],
    iterations = 20,
    algorithm_callback = StopMutationAfterNgenCallback(after_gen = 10),
    pop_size = 50,
    n_offsprings = 10,
    verbose = True
)


assert isinstance(result.algorithm.mating.mutation, NoMutation)

n_gen |  n_eval |  n_nds  |     eps      |  indicator  
    1 |      50 |      10 |            - |            -
    2 |      60 |      13 |  0.586146575 |        ideal
    3 |      70 |      11 |  0.013067766 |            f
    4 |      80 |       9 |  0.010056088 |            f
    5 |      90 |       9 |  0.007094227 |            f
    6 |     100 |      10 |  0.005057957 |            f
    7 |     110 |      10 |  0.003516311 |            f
    8 |     120 |      10 |  0.011673575 |            f
    9 |     130 |      10 |  0.011513896 |            f
   10 |     140 |      10 |  0.00000E+00 |            f
   11 |     150 |      12 |  0.026326012 |        ideal
   12 |     160 |      13 |  0.008444444 |        ideal
   13 |     170 |      14 |  0.013344303 |            f
   14 |     180 |      15 |  0.017438645 |        ideal
   15 |     190 |      16 |  0.015449438 |        ideal
   16 |     200 |      18 |  0.017350480 |        nadir
   17 |     210 |      18 |  0.012219959 |      